In [9]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import torch
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torchtuples as tt
from torchtuples import Model
from sklearn.model_selection import train_test_split

from dqAux import dqNetSparse,checkLoss,checkErrorMean,getSESingle

In [58]:
WXYgeneration(1000,5,2)

,W1,W2,W3,W4,W5,Z,Y,type
0,0.935362,0.752499,0.926603,0.770835,0.766542,1.234644,13.537398,2
1,0.389619,0.353367,0.039436,0.313640,0.258057,4.803607,5.450123,2
2,0.858532,0.840560,0.661594,0.250100,0.872319,2.118981,5.225932,2
3,0.858425,0.496159,0.594004,0.608602,0.389472,3.051821,4.550209,2
4,0.348896,0.502369,0.891981,0.398505,0.888788,1.866486,2.238722,2
...,...,...,...,...,...,...,...,...
995,0.897303,0.247983,0.620601,0.887230,0.772674,0.297475,2.109804,2
996,0.463706,0.349804,0.613407,0.331065,0.644575,3.769556,6.896632,2
997,0.503146,0.210500,0.169949,0.772159,0.517287,3.040146,3.969722,2
998,0.967825,0.772293,0.977470,0.221687,0.976156,1.546126,5.125076,2


In [63]:
df  = pd.read_csv("./data/concrete.csv")
df_train,df_test = train_test_split(df,test_size=0.2)

In [64]:
df.head(5)

,blst,flh,sup,cag,fag,age,inX,outY
0,0.0,0.0,2.5,1040.0,676.0,28,0.300000,4.381901
1,0.0,0.0,2.5,1055.0,676.0,28,0.300000,4.125358
2,142.5,0.0,0.0,932.0,594.0,270,0.685714,3.695607
3,142.5,0.0,0.0,932.0,594.0,365,0.685714,3.714791
4,132.4,0.0,0.0,978.4,825.5,360,0.966767,3.790984


In [61]:
df = WXYgeneration(1000,5,3)
df_train,df_test = train_test_split(df,test_size=0.2)

In [85]:
n = 1000
d = 5
typenum = 3
df = WXYgeneration(n,d,typenum)
df_train,df_val = train_test_split(df,test_size=0.2)
df_test = WXYgeneration(5000,d,typenum)

x_train_nonpar = torch.tensor(df_train.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_val_nonpar = torch.tensor(df_val.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_test_nonpar = torch.tensor(df_test.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))

x_train_lin = torch.tensor(df_train['Z'].values).view(len(df_train),1).type(torch.float32)
x_val_lin = torch.tensor(df_val['Z'].values).view(len(df_val),1).type(torch.float32)
x_test_lin = torch.tensor(df_test['Z'].values).view(len(df_test),1).type(torch.float32)

y_train = torch.tensor(df_train['Y'].values).view(len(df_train),1).type(torch.float32)
y_test = torch.tensor(df_test['Y'].values).view(len(df_test),1).type(torch.float32)
y_val = torch.tensor(df_val['Y'].values).view(len(df_val),1).type(torch.float32)

x_train = (x_train_lin,x_train_nonpar)
x_test = (x_test_lin,x_test_nonpar)
x_val = (x_val_lin,x_val_nonpar)
val_data = (x_val,y_val)
dim_lin,dim_nonpar=1,x_val_nonpar.shape[1]

# Create a LinearRegression model
model = LinearRegression()

In [111]:
# hyperparameters
batch_size = 128
nodes = [5,128] # number of network layers and nodes per layer
lr = 0.001
epochs = 500
verbose = False
sparseRatio = 0.5

In [112]:
# training Partially Linear Quantile Regression (DPLQR)
coef_init_weight = torch.tensor(paramsLin, dtype=y_train.dtype)
loss = nn.MSELoss()
model = Model(dqNetSparse(dim_lin, dim_nonpar, coef_init_weight, nodes, sparseRatio), loss)
model.optimizer.set_lr(lr)

In [66]:
df_train_lin = df_train
df_val = df_train.sample(frac=0.1)
df_train = df_train.drop(df_val.index)

In [67]:
# standardize covariates
cols_standardize = ['blst','flh','sup','cag','fag', 'age']
cols_leave = []

standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]
x_mapper = DataFrameMapper(standardize + leave)

x_train_nonpar = torch.tensor(x_mapper.fit_transform(df_train).astype('float32'))
x_val_nonpar = torch.tensor(x_mapper.transform(df_val).astype('float32'))
x_test_nonpar = torch.tensor(x_mapper.transform(df_test).astype('float32'))

x_train_lin = torch.tensor(df_train['inX'].values).view(len(df_train),1).type(torch.float32)
x_test_lin = torch.tensor(df_test['inX'].values).view(len(df_test),1).type(torch.float32)
x_val_lin = torch.tensor(df_val['inX'].values).view(len(df_val),1).type(torch.float32)

y_train = torch.tensor(df_train['outY'].values).view(len(df_train),1).type(torch.float32)
y_test = torch.tensor(df_test['outY'].values).view(len(df_test),1).type(torch.float32)
y_val = torch.tensor(df_val['outY'].values).view(len(df_val),1).type(torch.float32)

x_train = (x_train_lin,x_train_nonpar)
x_test = (x_test_lin,x_test_nonpar)
x_val = (x_val_lin,x_val_nonpar)
val_data = (x_val,y_val)

In [114]:
dim_lin,dim_nonpar=1,x_val_nonpar.shape[1]
tau_seq = [(i+1)/50 for i in range(49)]# quantile interval [0.02,0.98]
resTab = np.zeros((len(tau_seq),5))
resTabLin = np.zeros((len(tau_seq),5))
i=0

In [115]:
# training model with different quantile levels
for tau in tau_seq:
    # linear quantile regression (LQR)
    model_lin = smf.quantreg("outY~inX+blst+flh+sup+cag+fag+age",df_train_lin).fit(q=tau)
    paramsLin = model_lin.params[1]
    model_lin.bse
    model_lin.conf_int().loc['inX']
    # prediction for LQR
    pred_lin = model_lin.predict(df_test).values
    predErrLin = checkErrorMean(pred_lin.reshape(len(pred_lin),1),y_test.numpy(),tau=tau)
    
    # training Partially Linear Quantile Regression (DPLQR)
    coef_init_weight=torch.tensor(paramsLin,dtype=y_train.dtype)
    loss = checkLoss(tau=tau)
    model = Model(dqNetSparse(dim_lin,dim_nonpar,coef_init_weight,nodes,sparseRatio),loss)
    model.optimizer.set_lr(lr)
    callbacks = [tt.callbacks.EarlyStopping()]
    model.fit(x_train,y_train,batch_size,epochs,callbacks,verbose,val_data=val_data, val_batch_size=batch_size)
    weights = list(model.net.parameters())
    
    # calculate standard error
    x_resd = (torch.cat((x_train[0],x_val[0]),dim=0),torch.cat((x_train[1],x_val[1]),dim=0))
    resdErr = (torch.cat((y_train,y_val),dim=0) - model.predict(x_resd)).numpy()

    callbacks = [tt.callbacks.EarlyStopping()]
    # obtain asymptotic covariance matrix
    se = getSESingle(x_train,x_val,resdErr,tau,nodes,batch_size,lr,epochs, callbacks,verbose,logic=True)
    
    # prediction error (w.r.t. check loss) on test set
    preds = model.predict(x_test)
    check_loss_tau = checkErrorMean(preds.numpy(),y_test.numpy(),tau=tau)
    deepCoefEst=weights[0].detach().numpy()[0][0]
    
    intvLin = (model_lin.conf_int().loc['inX'][0],model_lin.conf_int().loc['inX'][1])
    s1=[tau,deepCoefEst,deepCoefEst-1.96*se[0],deepCoefEst+1.96*se[0],check_loss_tau]
    s2=[tau,paramsLin,intvLin[0],intvLin[1],predErrLin]
    
    resTab[i,:] = s1
    resTabLin[i,:] = s2
    i=i+1
    
resDeep = pd.DataFrame(resTab)
resDeep.columns = ['tau','estCoef','lowerCI','upperCI','CL']
resLin = pd.DataFrame(resTabLin)
resLin.columns = ['tau','estCoef','lowerCI','upperCI','CL']

/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/qxzhong/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:190: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [145]:
class ReLUNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ReLUNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Parameters
input_dim = 6  # Number of input features
output_dim = 1 # Number of output classes

# Instantiate the model
model = ReLUNetwork(input_dim, output_dim)

In [150]:
x_train_nonpar.shape

torch.Size([1600, 5])

In [152]:
torch.cat(( x_train_lin, x_train_nonpar), dim=1).shape

torch.Size([1600, 6])

In [159]:
val_data[0]

tensor([[-2.8947,  0.6169,  0.8434,  0.6519,  0.5547,  0.6499],
        [-0.6466,  0.1551,  0.1317,  0.1843,  0.0161,  0.1172],
        [ 1.0134,  0.6675,  0.7074,  0.0070,  0.6425,  0.6791],
        ...,
        [ 1.5189,  0.7990,  0.6055,  0.7107,  0.9629,  0.8916],
        [ 2.1646,  0.5227,  0.4137,  0.7606,  0.6730,  0.7360],
        [ 4.8441,  0.8400,  0.9705,  0.4800,  0.4622,  0.9127]])

In [160]:
dim_nonpar

5

In [177]:
df_train['Sum_First_5'] = df_train.iloc[:, :4].sum(axis=1)*2/5
df_train['new'] = df_train.iloc[:,5] + df_train.iloc[:,7]

In [178]:
df_train.head(5)

,W1,W2,W3,W4,W5,Z,Y,Sum_First_5,new
512,0.398103,0.391199,0.411176,0.566982,0.301867,2.971629,3.791888,0.706985,3.678613
1853,0.592168,0.071175,0.116200,0.351948,0.954468,3.268285,3.572499,0.452596,3.720882
1847,0.520641,0.419058,0.312510,0.180118,0.148076,0.174004,0.789175,0.572931,0.746934
1562,0.422780,0.478179,0.398474,0.130176,0.299166,2.443718,2.858257,0.571844,3.015561
958,0.412065,0.417453,0.751831,0.724861,0.541068,0.851595,1.993105,0.922484,1.774078


In [190]:

n = 2000
d = 5
typenum = 2
df = WXYgeneration(n,d,typenum)
df_train,df_val = train_test_split(df,test_size=0.2)
df_test = WXYgeneration(5000,d,typenum)

x_train_nonpar = torch.tensor(df_train.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_val_nonpar = torch.tensor(df_val.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_test_nonpar = torch.tensor(df_test.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))

x_train_lin = torch.tensor(df_train['Z'].values).view(len(df_train),1).type(torch.float32)
x_val_lin = torch.tensor(df_val['Z'].values).view(len(df_val),1).type(torch.float32)
x_test_lin = torch.tensor(df_test['Z'].values).view(len(df_test),1).type(torch.float32)

y_train = torch.tensor(df_train['Y'].values).view(len(df_train),1).type(torch.float32)
y_test = torch.tensor(df_test['Y'].values).view(len(df_test),1).type(torch.float32)
y_val = torch.tensor(df_val['Y'].values).view(len(df_val),1).type(torch.float32)

x_train = torch.cat(( x_train_lin, x_train_nonpar), dim=1)
x_test = torch.cat((x_test_lin,x_test_nonpar), dim=1)
x_val = torch.cat((x_val_lin,x_val_nonpar), dim=1)
val_data = (x_val,y_val)
dim_lin,dim_nonpar=1,x_val_nonpar.shape[1]

class ReLUNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ReLUNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# hyperparameters
batch_size = 128
nodes = [5,50] # number of network layers and nodes per layer
lr = 0.001
epochs = 500
verbose = 2
sparseRatio = 0
input_dim = 6
output_dim = 1

# training Partially Linear Quantile Regression (DPLQR)
coef_init_weight = torch.tensor(paramsLin, dtype=y_train.dtype)
loss = nn.MSELoss()
#model = Model(dqNetSparse(dim_lin, dim_nonpar, coef_init_weight, nodes, sparseRatio), loss)
model = Model(ReLUNetwork(input_dim, output_dim), loss)
model.optimizer.set_lr(lr)
callbacks = [tt.callbacks.EarlyStopping()]
model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val_data, val_batch_size=5*batch_size)
weights = list(model.net.parameters())

deepCoefEst=weights[0].detach().numpy()[0][0]

0:	[0s / 0s],		train_loss: 150.0246,	val_loss: 52.7384
1:	[0s / 0s],		train_loss: 88.5921,	val_loss: 14.8934
2:	[0s / 0s],		train_loss: 23.8643,	val_loss: 9.4710
3:	[0s / 0s],		train_loss: 21.3073,	val_loss: 11.1004
4:	[0s / 0s],		train_loss: 24.2141,	val_loss: 5.8946
5:	[0s / 0s],		train_loss: 13.9625,	val_loss: 5.9254
6:	[0s / 0s],		train_loss: 25.8824,	val_loss: 4.3037
7:	[0s / 0s],		train_loss: 21.6952,	val_loss: 3.2539
8:	[0s / 0s],		train_loss: 7.1024,	val_loss: 3.0420
9:	[0s / 0s],		train_loss: 6.6525,	val_loss: 2.8563
10:	[0s / 0s],		train_loss: 6.4938,	val_loss: 2.8724
11:	[0s / 0s],		train_loss: 6.4826,	val_loss: 2.8980
12:	[0s / 0s],		train_loss: 6.5557,	val_loss: 2.8182
13:	[0s / 0s],		train_loss: 6.4371,	val_loss: 2.7862
14:	[0s / 0s],		train_loss: 6.4152,	val_loss: 2.7917
15:	[0s / 0s],		train_loss: 6.4738,	val_loss: 2.8319
16:	[0s / 0s],		train_loss: 6.9493,	val_loss: 3.0333
17:	[0s / 0s],		train_loss: 7.2641,	val_loss: 2.9665
18:	[0s / 0s],		train_loss: 9.8323,	val_loss

In [263]:

n = 2000
d = 5
typenum = 3
df = WXYgeneration(n,d,typenum)
df_train,df_val = train_test_split(df,test_size=0.2)
df_test = WXYgeneration(5000,d,typenum)

x_train_nonpar = torch.tensor(df_train.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_val_nonpar = torch.tensor(df_val.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_test_nonpar = torch.tensor(df_test.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))

x_train_lin = torch.tensor(df_train['Z'].values).view(len(df_train),1).type(torch.float32)
x_val_lin = torch.tensor(df_val['Z'].values).view(len(df_val),1).type(torch.float32)
x_test_lin = torch.tensor(df_test['Z'].values).view(len(df_test),1).type(torch.float32)

y_train = torch.tensor(df_train['Y'].values).view(len(df_train),1).type(torch.float32)
y_test = torch.tensor(df_test['Y'].values).view(len(df_test),1).type(torch.float32)
y_val = torch.tensor(df_val['Y'].values).view(len(df_val),1).type(torch.float32)

x_train = (x_train_lin,x_train_nonpar)
x_test = (x_test_lin,x_test_nonpar)
x_val = (x_val_lin,x_val_nonpar)
val_data = (x_val,y_val)
dim_lin,dim_nonpar=1,x_val_nonpar.shape[1]

# Create a LinearRegression model
model = LinearRegression()

# Train the model
model.fit(df_train.drop(columns=['Y']), df_train['Y'])
paramsLin = model.coef_[-1]

# hyperparameters
batch_size = 128
nodes = [5,128] # number of network layers and nodes per layer
lr = 0.001
epochs = 500
verbose = 2
sparseRatio = 0.5


# training Partially Linear Quantile Regression (DPLQR)
coef_init_weight = torch.tensor(paramsLin, dtype=y_train.dtype)
loss = nn.MSELoss()
model = Model(dqNetSparse(dim_lin, dim_nonpar, coef_init_weight, nodes, sparseRatio), loss)
model.optimizer.set_lr(lr)
callbacks = [tt.callbacks.EarlyStopping()]
model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val_data, val_batch_size=5*batch_size)
weights = list(model.net.parameters())

deepCoefEst=weights[0].detach().numpy()[0][0]

0:	[0s / 0s],		train_loss: 3.1037,	val_loss: 4.5559
1:	[0s / 0s],		train_loss: 3.0884,	val_loss: 4.4368
2:	[0s / 0s],		train_loss: 2.9772,	val_loss: 4.1398
3:	[0s / 0s],		train_loss: 2.7377,	val_loss: 3.9429
4:	[0s / 0s],		train_loss: 2.6027,	val_loss: 3.8321
5:	[0s / 0s],		train_loss: 2.5182,	val_loss: 3.9388
6:	[0s / 0s],		train_loss: 2.5933,	val_loss: 3.7746
7:	[0s / 0s],		train_loss: 2.5274,	val_loss: 3.7790
8:	[0s / 0s],		train_loss: 2.4759,	val_loss: 3.7444
9:	[0s / 0s],		train_loss: 2.6120,	val_loss: 3.7712
10:	[0s / 0s],		train_loss: 2.4711,	val_loss: 3.8690
11:	[0s / 0s],		train_loss: 2.4683,	val_loss: 3.7472
12:	[0s / 0s],		train_loss: 2.5361,	val_loss: 3.7624
13:	[0s / 0s],		train_loss: 2.4453,	val_loss: 3.7729
14:	[0s / 0s],		train_loss: 2.4503,	val_loss: 3.8022
15:	[0s / 0s],		train_loss: 2.4429,	val_loss: 3.7433
16:	[0s / 1s],		train_loss: 2.5305,	val_loss: 3.8823
17:	[0s / 1s],		train_loss: 2.4471,	val_loss: 3.7380
18:	[0s / 1s],		train_loss: 2.4299,	val_loss: 3.7308
19:

In [264]:
paramsLin

0.993858859666338

In [265]:
deepCoefEst

0.99495703

In [267]:
deepCoefEst, model = g_deepfit(df_train,x_train,y_train,dim_lin,dim_nonpar,sparseRatio,val_data,nodes,batch_size,lr,epochs,verbose)
z_delta = m_deepfit(x_train,x_val,nodes,batch_size,lr,epochs,verbose)


0:	[0s / 0s],		train_loss: 3.2085,	val_loss: 4.6129
1:	[0s / 0s],		train_loss: 3.0598,	val_loss: 4.4416
2:	[0s / 0s],		train_loss: 2.9693,	val_loss: 4.1277
3:	[0s / 0s],		train_loss: 2.7532,	val_loss: 3.9560
4:	[0s / 0s],		train_loss: 2.6600,	val_loss: 3.8329
5:	[0s / 0s],		train_loss: 2.5708,	val_loss: 3.7600
6:	[0s / 0s],		train_loss: 2.5509,	val_loss: 3.7685
7:	[0s / 0s],		train_loss: 2.4854,	val_loss: 3.7566
8:	[0s / 0s],		train_loss: 2.4703,	val_loss: 3.7664
9:	[0s / 0s],		train_loss: 2.4704,	val_loss: 3.8308
10:	[0s / 0s],		train_loss: 2.5889,	val_loss: 3.8728
11:	[0s / 0s],		train_loss: 2.4797,	val_loss: 3.7596
12:	[0s / 0s],		train_loss: 2.4948,	val_loss: 3.7683
13:	[0s / 0s],		train_loss: 2.4612,	val_loss: 3.7302
14:	[0s / 0s],		train_loss: 2.4697,	val_loss: 3.7228
15:	[0s / 1s],		train_loss: 2.4579,	val_loss: 3.7296
16:	[0s / 1s],		train_loss: 2.4749,	val_loss: 3.7843
17:	[0s / 1s],		train_loss: 2.4540,	val_loss: 3.8122
18:	[0s / 1s],		train_loss: 2.4695,	val_loss: 3.8429
19:

In [273]:
z_merge = z_merge.numpy()

In [271]:
z_delta,z_merge = m_deepfit(x_train,x_val,nodes,batch_size,lr,epochs,verbose)

0:	[0s / 0s],		train_loss: 23.6258,	val_loss: 28.7208
1:	[0s / 0s],		train_loss: 23.8408,	val_loss: 28.7272
2:	[0s / 0s],		train_loss: 23.5243,	val_loss: 28.7794
3:	[0s / 0s],		train_loss: 23.5910,	val_loss: 28.7818
4:	[0s / 0s],		train_loss: 23.4791,	val_loss: 28.7822
5:	[0s / 0s],		train_loss: 23.5371,	val_loss: 28.7976
6:	[0s / 0s],		train_loss: 23.6329,	val_loss: 28.7847
7:	[0s / 0s],		train_loss: 23.6090,	val_loss: 28.8232
8:	[0s / 0s],		train_loss: 23.5294,	val_loss: 28.8349
9:	[0s / 0s],		train_loss: 23.5250,	val_loss: 28.7627
10:	[0s / 0s],		train_loss: 23.5845,	val_loss: 28.8713


In [281]:
y_delta = (y_merge - preds).numpy()

NameError: name 'preds' is not defined

In [285]:
y_delta = (y_merge - preds).numpy()

In [299]:
def WXgeneration(n,d,typenum):
    W = np.random.uniform(0,1,size = n)
    # Generate error terms 
    v = stats.t.rvs(df=2, loc=0, scale=1, size=n)
    if typenum == 1:
        Z = np.array([m1(w) for w in W]) + v
    elif typenum == 2:
        Z = np.array([m2(w) for w in W]) + v
    elif typenum == 3:
        Z = np.array([m3(w) for w in W]) + v
    else:
        raise ValueError("Invalid type. Please choose type 1, 2, or 3.")

    # Create DataFrame
    df = pd.DataFrame(W, columns=[f"W{j}" for j in range(1, d + 1)])
    df['Z'] = Z

    return df

In [306]:

def WXgeneration(n,d,typenum):
    W = np.random.uniform(0,1,size = (n,d))
    # Generate error terms
    v = stats.t.rvs(df=2, loc=0, scale=1, size=n)
    if typenum == 1:
        Z = np.array([m1(w) for w in W]) + v
    elif typenum == 2:
        Z = np.array([m2(w) for w in W]) + v
    elif typenum == 3:
        Z = np.array([m3(w) for w in W]) + v
    else:
        raise ValueError("Invalid type. Please choose type 1, 2, or 3.")

    # Create DataFrame
    df = pd.DataFrame(W, columns=[f"W{j}" for j in range(1, d + 1)])
    df['Z'] = Z

    return df

In [307]:
df_unlabel = WXgeneration(n,d,typenum)

In [308]:
df_unlabel

,W1,W2,W3,W4,W5,Z
0,0.182126,0.705036,0.605630,0.577382,0.515087,-0.103836
1,0.073311,0.717807,0.152682,0.420476,0.720257,-0.824600
2,0.131697,0.995636,0.487351,0.262571,0.636707,-0.033877
3,0.036318,0.047510,0.522354,0.821338,0.861367,-0.146784
4,0.508992,0.492400,0.276370,0.465576,0.024046,3.269712
...,...,...,...,...,...,...
1995,0.424737,0.287737,0.620835,0.611959,0.266815,1.411615
1996,0.640607,0.233286,0.657402,0.976497,0.143034,-0.220037
1997,0.287611,0.343185,0.069774,0.164646,0.846802,-0.062164
1998,0.899277,0.393960,0.649641,0.185577,0.433956,-0.118594


In [312]:
pd.concat([df, df_unlabel],axis = 0)

,W1,W2,W3,W4,W5,Z,Y
0,0.028288,0.063519,0.146434,0.099195,0.113721,-0.940675,1.980789
1,0.346075,0.889532,0.355796,0.753103,0.859630,-0.269944,-1.128505
2,0.228886,0.950133,0.759375,0.712147,0.829649,-0.137361,-1.057112
3,0.977934,0.935355,0.727733,0.817390,0.628250,2.243296,2.825422
4,0.918715,0.591772,0.658769,0.393925,0.917530,1.117078,-0.525148
...,...,...,...,...,...,...,...
1995,0.424737,0.287737,0.620835,0.611959,0.266815,1.411615,NaN
1996,0.640607,0.233286,0.657402,0.976497,0.143034,-0.220037,NaN
1997,0.287611,0.343185,0.069774,0.164646,0.846802,-0.062164,NaN
1998,0.899277,0.393960,0.649641,0.185577,0.433956,-0.118594,NaN


In [315]:
n_unlabel = n*int(math.sqrt(n))
df_unlabel = WXgeneration(n,d,typenum)
df_ss = pd.concat([df, df_unlabel],axis = 0)
df_ss_train,df_ss_val = train_test_split(df_ss,test_size=0.2)

x_ss_train_nonpar = torch.tensor(df_ss_train.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_ss_val_nonpar = torch.tensor(df_ss_val.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_ss_train_lin = torch.tensor(df_ss_train['Z'].values).view(len(df_ss_train),1).type(torch.float32)
x_ss_val_lin = torch.tensor(df_ss_val['Z'].values).view(len(df_ss_val),1).type(torch.float32)

x_ss_train = (x_ss_train_lin,x_ss_train_nonpar)
x_ss_val = (x_val_lin,x_val_nonpar)

z_ss_delta,z_ss_merge,model_ss_m = m_deepfit(x_ss_train,x_ss_val,nodes,batch_size,lr,epochs,verbose)

0:	[0s / 0s],		train_loss: 15.3620,	val_loss: 28.7969
1:	[0s / 0s],		train_loss: 15.3476,	val_loss: 28.7446
2:	[0s / 0s],		train_loss: 15.3454,	val_loss: 28.7588
3:	[0s / 0s],		train_loss: 15.3388,	val_loss: 28.7215
4:	[0s / 0s],		train_loss: 15.3329,	val_loss: 28.7782
5:	[0s / 0s],		train_loss: 15.3386,	val_loss: 28.7486
6:	[0s / 0s],		train_loss: 15.3218,	val_loss: 28.8142
7:	[0s / 0s],		train_loss: 15.3286,	val_loss: 28.7628
8:	[0s / 0s],		train_loss: 15.3322,	val_loss: 28.7759
9:	[0s / 0s],		train_loss: 15.3170,	val_loss: 28.7858
10:	[0s / 0s],		train_loss: 15.3028,	val_loss: 28.8124
11:	[0s / 0s],		train_loss: 15.3098,	val_loss: 28.9090
12:	[0s / 0s],		train_loss: 15.2834,	val_loss: 28.7524
13:	[0s / 1s],		train_loss: 15.2962,	val_loss: 28.8292


In [318]:
z_ss_delta,z_ss_merge,model_ss_m = m_deepfit(x_ss_train,x_ss_val,nodes,batch_size,lr,epochs,verbose)
x_labeled = torch.tensor(df.drop(columns=['Z','Y']).to_numpy().astype('float32'))
z_labeled = torch.tensor(df['Z'].values).view(len(df),1).type(torch.float32)
z_labeled_delta = (z_labeled - model_ss_m.predict(x_labeled)).numpy()


0:	[0s / 0s],		train_loss: 15.3588,	val_loss: 28.7238
1:	[0s / 0s],		train_loss: 15.3438,	val_loss: 28.7450
2:	[0s / 0s],		train_loss: 15.3371,	val_loss: 28.7619
3:	[0s / 0s],		train_loss: 15.3390,	val_loss: 28.8072
4:	[0s / 0s],		train_loss: 15.3226,	val_loss: 28.8174
5:	[0s / 0s],		train_loss: 15.3259,	val_loss: 28.8193
6:	[0s / 0s],		train_loss: 15.3261,	val_loss: 28.7621
7:	[0s / 0s],		train_loss: 15.3042,	val_loss: 28.8158
8:	[0s / 0s],		train_loss: 15.3015,	val_loss: 28.8835
9:	[0s / 0s],		train_loss: 15.3062,	val_loss: 28.8382
10:	[0s / 0s],		train_loss: 15.2934,	val_loss: 28.8399


In [319]:
z_labeled_delta.shape

(2000, 1)

In [298]:
deepCoefEst

0.9946609

In [297]:
DMLCoefEst

1.0015007

In [326]:
DMLCoefEst =   np.mean(z_delta * y_delta) / np.mean(z_delta * z_merge)

In [346]:
verbose = 0

In [347]:
n_unlabel = n*int(math.sqrt(n))
df_unlabel = WXgeneration(n,d,typenum)
df_ss = pd.concat([df, df_unlabel],axis = 0)
df_ss_train,df_ss_val = train_test_split(df_ss,test_size=0.2)

x_ss_train_nonpar = torch.tensor(df_ss_train.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_ss_val_nonpar = torch.tensor(df_ss_val.drop(columns=['Z', 'Y']).to_numpy().astype('float32'))
x_ss_train_lin = torch.tensor(df_ss_train['Z'].values).view(len(df_ss_train),1).type(torch.float32)
x_ss_val_lin = torch.tensor(df_ss_val['Z'].values).view(len(df_ss_val),1).type(torch.float32)

x_ss_train = (x_ss_train_lin,x_ss_train_nonpar)
x_ss_val = (x_val_lin,x_val_nonpar)

z_ss_delta,z_ss_merge,model_ss_m = m_deepfit(x_ss_train,x_ss_val,nodes,batch_size,lr,epochs,verbose)
x_labeled = torch.tensor(df.drop(columns=['Z','Y']).to_numpy().astype('float32'))
z_labeled = torch.tensor(df['Z'].values).view(len(df),1).type(torch.float32)
z_labeled_delta = (z_labeled - model_ss_m.predict(x_labeled)).numpy()
z_labeled = z_labeled.numpy()

In [429]:
df_train, df_val = train_test_split(df, test_size=0.2)
model = LinearRegression()

# Train the model
model.fit(df_train.drop(columns=['Y']), df_train['Y'])
paramsLin = model.coef_[-1]
paramsLin

1.2265478266723342

In [440]:
n = 500
d = 15
typenum = 3

In [443]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [444]:
device

device(type='cpu')

In [441]:
paramsLin, deepCoefEst, DMLCoefEst, SSDMLCoefEst = Single_trial(n,d,typenum,batch_size,nodes,lr,epochs,verbose,sparseRatio)

In [442]:
print(paramsLin)
print(deepCoefEst)
print(DMLCoefEst)
print(SSDMLCoefEst)

1.8078729684097332
1.7579889
0.75270617
0.78333914


In [343]:
x_labeled_train = x_train[1]
x_labeled_val = x_val[1]
x_labeled = torch.cat((x_labeled_train,x_labeled_val),dim=0)
z_labeled_train = (x_train[0]).view(x_train[0].size()[0],1)
z_labeled_val = (x_val[0]).view(x_val[0].size()[0],1)
z_labeled = torch.cat((z_labeled_train,z_labeled_val),dim=0)
z_labeled_delta = (z_labeled - model_ss_m.predict(x_labeled)).numpy()
z_labeled = z_labeled.numpy()
SSDMLCoefEst =   np.mean(z_labeled_delta * y_delta) / np.mean(z_labeled_delta * z_labeled)


In [344]:
SSDMLCoefEst

1.0058291

In [336]:
z_merge[1:10,:]

array([[-0.01936181],
       [-2.956221  ],
       [-0.7126476 ],
       [ 1.0401571 ],
       [-0.4676155 ],
       [ 3.6208308 ],
       [ 0.39294863],
       [-0.74710864],
       [-0.46995282]], dtype=float32)

In [333]:
z_merge - z_labeled

array([[ 0.4480143 ],
       [ 0.2505819 ],
       [-2.8188596 ],
       ...,
       [-1.8950634 ],
       [ 0.882874  ],
       [ 0.79095995]], dtype=float32)

In [116]:
# confidence intervals area via Riemann sum
delta = tau_seq[1]-tau_seq[0]
CIArea = ((resDeep['upperCI'] - resDeep['lowerCI'])*delta).sum()
CIAreaLin = ((resLin['upperCI'] - resLin['lowerCI'])*delta).sum()
print("95% CI Area under DPLQR:",CIArea)
print("95% CI Area under LQR:",CIAreaLin)

95% CI Area under DPLQR: 0.2223621286694169
95% CI Area under LQR: 0.3364240113753357


In [117]:
# ACL
acl = (resDeep['CL']*delta).sum()
aclLin = (resLin['CL']*delta).sum()
print('ACL under DPLQR:',acl)
print('ACL under LQR:',aclLin)

ACL under DPLQR: 0.058949435222893955
ACL under LQR: 0.09680439909407373


In [118]:
# save for plot
resDeep.to_csv('./data/resDeep.csv')
resLin.to_csv('./data/resLin.csv')

In [119]:
# save train and test data for PLAQR
# which is implemented by R package plaqr
df_train.to_csv("./data/df_train.csv")
df_test.to_csv("./data/df_test.csv")